In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

df = pd.read_csv('events_up_to_01062018.csv')
pd.options.mode.chained_assignment = None

/home/amaherok/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Me quedo solo con las columnas que son de interés para este análisis

In [2]:
df_tiempos=  df[['timestamp','person']]
df_tiempos['timestamp'] = pd.to_datetime(df_tiempos['timestamp'])
df_tiempos= df_tiempos.sort_values(by=["timestamp"])
df_tiempos['just_date'] = df_tiempos['timestamp'].dt.date
df_tiempos.reset_index(drop = True, inplace = True)

Agrego la columna **'diff'** que me indica _diferencia de tiempo entre los distintos eventos temporales **por usuario**_

In [3]:
df_tiempos['diff'] = df_tiempos.groupby(['person'])['timestamp'].diff()

Paso la columna a string para detectar los valores "NaT" que indican el ingreso de un nuevo usuario 

In [4]:
df_tiempos['diff'] = df_tiempos['diff'].astype(str)
df_tiempos['new_user'] = df_tiempos['diff'] =='NaT'

Agrego la columna **'diff2'** que me indica la _diferencia entre eventos temporales_ (sin importar el usuario)

In [5]:
df_tiempos['diff2'] = (df_tiempos['timestamp'] - (df_tiempos['timestamp'].shift())) / np.timedelta64(1, 'h')

Reasigno la columna **'diff'** para volver a tener datos de tipo temporal y elimina los nulos

In [6]:
df_tiempos['diff'] = df_tiempos.groupby(['person'])['timestamp'].diff()

Lleno con 0s todos los nulos del data frame

In [7]:
df_tiempos=df_tiempos.fillna(0)

Identifico en la columna **'new_session_same_user'** los _comienzos de nuevas sesiones del mismo usuario_. Para ello se tiene en cuenta un umbral de 0.48 horas. Es decir, se considera que si el tiempo entre eventos es mayor a 0.48 horas, el evento corresponde a una nueva sesion del mismo usuario. En el informe adjunto se explica detalladamente la elección de este umbral.

In [8]:
df_tiempos['new_session_same_user'] = df_tiempos['diff2'] > 0.48

Identifico en la columna **'new_session_new_user'** los comienzos de _nuevas sesiones de un usuario nuevo._ Se tiene en cuenta el mismo umbral que antes. 

In [9]:
df_tiempos["diff"]=df_tiempos["diff"]/np.timedelta64(1, 'h')
df_tiempos["new_session_new_user"] = df_tiempos["diff"] > 0.48

Para contabilizar las sesiones nuevas (que pueden corresponder al mismo usuario o a un nuevo usuario) se realiza la operacion OR entre las columnas con datos booleanos calculadas previamente.

In [10]:
df_tiempos["new_session"]=df_tiempos["new_user"]|df_tiempos["new_session_same_user"]|df_tiempos["new_session_new_user"]

Enumero las sesiones para luego separarlas por agrupación

In [11]:
df_tiempos['sessionid'] = df_tiempos['new_session'].cumsum()

Como pusimos un umbral de 0.48 horas para la finalizacion de las sesiones, elimino de la columa **'diff'** los valores que superen ese valor (Me quedo solo con los False de la columna new_sesion). Estos valores se corresponderan al primer 'diff' de cada sesion, que debería ser siempre nulo.

In [12]:
df_tiempos=df_tiempos[df_tiempos.new_session==False]

Ahora si agrupo por **sessionid** y obtengo la información buscada

In [13]:
df_tiempos.head()

,timestamp,person,just_date,diff,new_user,diff2,new_session_same_user,new_session_new_user,new_session,sessionid
1,2018-01-01 08:09:31,0f4e2a4b,2018-01-01,0.000000,False,0.000000,False,False,False,1
2,2018-01-01 08:09:31,0f4e2a4b,2018-01-01,0.000000,False,0.000000,False,False,False,1
3,2018-01-01 08:09:44,0f4e2a4b,2018-01-01,0.003611,False,0.003611,False,False,False,1
5,2018-01-01 08:45:29,0f4e2a4b,2018-01-01,0.000000,False,0.000000,False,False,False,2
6,2018-01-01 08:45:29,0f4e2a4b,2018-01-01,0.000000,False,0.000000,False,False,False,2


In [14]:
tiempos_sesiones=df_tiempos.groupby(["sessionid"]).agg({'diff':'sum','person':'first'}).reset_index()

In [15]:
tiempos_sesiones.head()

,sessionid,diff,person
0,1,0.003611,0f4e2a4b
1,2,0.070556,0f4e2a4b
2,3,0.000000,7c7e0de9
3,4,1.136111,8af11dbc
4,5,0.024167,23252ece


In [23]:
session_mean = tiempos_sesiones.groupby(["person"]).agg({'diff':'mean'}).reset_index()

In [24]:
session_mean.head()

,person,diff
0,0008ed71,0.001389
1,00091926,0.142760
2,00091a7a,0.171111
3,000ba417,0.128333
4,000c79fe,0.184444


In [25]:
session_mean = df['person'].drop_duplicates().to_frame().merge(session_mean , left_on='person', right_on='person', how='left')
session_mean.shape

(38829, 2)

In [27]:
session_mean.to_csv('diff.csv', index=False)

## XgBoost

In [18]:
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

In [19]:
df_train = df_labels.merge(session_mean , left_on='person', right_on='person', how='left')

In [20]:
df_train.head()

,person,label,diff
0,0566e9c1,0,0.059618
1,6ec7ee77,0,NaN
2,abe7a2fb,0,0.095119
3,34728364,0,0.043148
4,87ed62de,0,0.155278


In [21]:
X, y = df_train.iloc[:,2].to_frame(),df_train.iloc[:,1]
X.head()

,diff
0,0.059618
1,NaN
2,0.095119
3,0.043148
4,0.155278


In [22]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [23]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [62]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 4,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 1500) #1200 era la mejro

In [63]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)


In [64]:
xg_reg.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=2000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [65]:
preds = xg_reg.predict(X_test)

In [66]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.212498


In [67]:
persons = df_labels['person']
df_predict = session_mean.loc[~session_mean.person.isin(persons)]
ppl_to_predict = (df.loc[~df['person'].isin(persons)])['person'].to_frame()
ppl_to_predict = ppl_to_predict.drop_duplicates('person')
df_predict = ppl_to_predict.merge(df_predict, left_on = 'person' , right_on = 'person', how='left')
X_predict = df_predict.drop(['person'], axis=1)

In [68]:
entrie = xg_reg.predict(X_predict)

In [69]:
seriesita = pd.Series(entrie)

In [70]:
df_entrie = df_predict['person'].to_frame()
df_entrie['label'] = seriesita

In [71]:
df_entrie.head()

,person,label
0,4886f805,0.047327
1,0297fc1e,0.081598
2,2d681dd8,0.067482
3,cccea85e,0.063965
4,4c8a8b93,0.067482


In [72]:
df_entrie = df_entrie.fillna(0)
df_entrie.head()

,person,label
0,4886f805,0.047327
1,0297fc1e,0.081598
2,2d681dd8,0.067482
3,cccea85e,0.063965
4,4c8a8b93,0.067482


In [73]:
df_entrie.to_csv(path_or_buf = 'entrie2.0', index = False)

In [74]:
df_entrie.shape

(19415, 2)

In [75]:
df_entrie['label'].nlargest(1)

5267    0.310678
Name: label, dtype: float32